In [ ]:


from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  df = pd.read_csv(io.BytesIO(uploaded[fn]))

# Check for duplicate rows
duplicate_rows = df[df.duplicated()]

if not duplicate_rows.empty:
  print("DataFrame contains duplicate rows:")
  print(duplicate_rows)
else:
  print("DataFrame does not contain duplicate rows.")

Saving seilamang.csv to seilamang (1).csv
User uploaded file "seilamang (1).csv" with length 3872 bytes
DataFrame does not contain duplicate rows.


In [ ]:
X = dados.drop(columns=['continuar'])  # Remova a coluna "continuar" das features
y = dados['continuar']                # Essa é a variável alvo


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Criar e treinar o modelo Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Fazer previsões
y_pred = random_forest.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Importâncias das características
importances = random_forest.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f'{feature}: {importance:.2f}')

Acurácia: 0.33

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.25      0.50      0.33         2
           1       0.50      0.25      0.33         4

    accuracy                           0.33         6
   macro avg       0.38      0.38      0.33         6
weighted avg       0.42      0.33      0.33         6


Matriz de Confusão:
[[1 1]
 [3 1]]
Score: 0.21
Perfil do Jogador: 0.13
Aposta para continuar: 0.15
Jackpot: 0.30
Jogadores Restantes: 0.07
Desistentes: 0.14
